# Chapter 5 - Part 2: Deploy UDF to Snowflake via Azure Functions

Welcome to chapter 5 of our Snowflake Data Scientist training series.

In chapter 5 we will look at model deployment options. The code is structured into three parts:
- Part 1: We train a model and save its binary file to disk, called the "pickle file".
- Part 2: We deploy the pickle file to an Azure Function and call it via API Integration from Snowflake
- Part 3: We deploy the pickle file directly to Snowflake using SnowPark.

Happy coding!


### 0.) Background

Now that we have our model generated and stored as pickle file, we need to start serving it back to Snowflake. 
Typically you'd want to do this via some kind of HTTP interface that can be called.

Any Flask application will do the job here, hosted on any of the three big cloud providers (and only the three, as we need the API integration). In this notebook we will exemplarily do this for Azure. If you are an AWS user, have a look at the Snowflake GitHUB repo: https://github.com/Snowflake-Labs/sfguide-external-functions-examples

![](imgs/external-functions-overview-07.png)

(Image copyright Snowflake: https://docs.snowflake.com/en/sql-reference/external-functions-introduction.html)

### 1.) Development of HTTP function that reacts locally

See here for our Azure function HTTP code. It reacts to HTTP requests and returns our model inference.

In [ ]:
import json
import joblib
import logging

from sklearn.ensemble import RandomForestRegressor

import azure.functions as func


filename = 'BPPredictor/randomforest_classifier.joblib.pkl'
rfr = joblib.load(filename)

def main(req: func.HttpRequest) -> func.HttpResponse:

    data = req.params.get('data')
    if not data:
        try:
            req_body = req.get_json()
        except ValueError:
            pass
        else:
            data = req_body.get('data')

    if data:

        retVal = []
        for i, val in enumerate(data):

            ## Two arguments in our call expected, sf_ext_fct(sex_agrgrp_position, bp_before)
            sex_agrgrp_position = val[1]
            bp_before = val[2]

            retVal.append([i, rfr.predict([[sex_agrgrp_position, bp_before]])[0]])
    
        ## See here for format: https://docs.snowflake.com/en/sql-reference/external-functions-data-format.html
        return func.HttpResponse(
            json.dumps({"data": retVal}),
            mimetype = 'application/json',
            charset = 'utf-8'
        )

    else:
        return func.HttpResponse(
             "Please pass data in the request body.",
             status_code = 400
        )


It is easiest to generate the code above using Visual Studio with Azure plugin.
![](imgs/azure-fct-setup.png)

After creation of your function, you can locally develop and debug till you are happy with the result.

![](imgs/azure-fct-local-run.png)

### 2.) Deploy to Azure & add API Management

You have two options to deploy your function to Azure now.
- Option 1: Via Visual Studio code
- Option 2: Via Github Actions for continous deployment.

Both options are very well described by Snowflake here, https://docs.snowflake.com/en/sql-reference/external-functions-creating-azure-ui.html.

Following the deployment, your application will be available via HTTP. In a next step you have to deploy an API Management 
solution which takes care of authentication, rate limiting and integration with Snowflake. Again this is very well described here, https://docs.snowflake.com/en/sql-reference/external-functions-creating-azure-ui.html.

### 3.) Configure Snowflake for inference
Once your model is deployed to a lamba/function service, you need to configure the API integration with Snowflake. For Azure, this works as below. You'll need your tenant id and the application id, which will be created by the API management service.

Make sure to call **describe** on the created api integration object to authenticate the service. If your company setting doesn't allow for this, check for api_key parameters.

In [ ]:
%%sql
create or replace api integration az_function_integration
    api_provider = azure_api_management
    azure_tenant_id = '<tenant-id>'
    azure_ad_application_id = '<azure-ad-application-id>'
    api_allowed_prefixes = ('https://<endpoint>.azure-api.net/')
    enabled = true;
    
--- Look out for consent URL here and click it 
describe api integration az_function_integration;

Now that the API integration is created, we can start creating our external function.

In [ ]:
create or replace external function sf_ext_fct_bp_after_predictor(parameter1 int)
    returns varchar
    api_integration = az_function_integration
    as 'https://<endpoint>.azure-api.net/<endpoint-url>';

Finally give it a call and see if everything works.

In [ ]:
select sf_ext_fct_bp_after_predictor(0, 155);

### 4.) Enjoy :)
Now that we are done deploying the model, we can use it in our pipeline.